In [1]:
import pandas as pd

2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [2]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')


3. To create the above dataframe:

    The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [3]:
df_table = data[0]

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    

In [4]:
df_table = df_table[df_table['Borough'] != 'Not assigned']

In [5]:
df_table.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

--code in cell below demonstrates this. It prints the value if there are any duplicates. In this case, nothing is printed.

In [6]:
for value in df_table.groupby('Postal Code').count().Borough:
    if value != 1:
        print(value)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
    Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

 No assumptions are made because we checked every value this way

In [7]:
df_table.shape

(103, 3)

loads the geo data from csv 

In [8]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')

merge the dataframes along postal codes

In [9]:
df = pd.merge(df_table, df_geo, on='Postal Code')
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


import the packages we'll need for mapping and clustering

In [10]:
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
import folium
import requests

create user_agent

In [11]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent='toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates for Toronto are ' + str(latitude) + ', ' + str(longitude))

Coordinates for Toronto are 43.6534817, -79.3839347


Create map using df

In [12]:
map_toronto = folium.Map(location=[latitude, longitude])
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng], radius=5, popup=label, color='green').add_to(map_toronto)
map_toronto
    

Now we create another map, but only take points where the Borough name includes 'Toronto'. Creates df_toronto to hold these lines

In [22]:
columns = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']
df_toronto = pd.DataFrame(columns=columns)

map_toronto = folium.Map(location=[latitude, longitude])
for line, borough in enumerate(df['Borough']):
    if 'Toronto' in borough:
        label = '{}, {}'.format(df['Neighborhood'][line], df['Borough'][line])
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker([df['Latitude'][line],df['Longitude'][line]], radius=5, popup=label, color='green').add_to(map_toronto)
        df_toronto = df_toronto.append(df.loc[line])
df_toronto = df_toronto.reset_index(drop=True)
map_toronto

In [23]:
df_toronto.head()

,Borough,Neighborhood,Latitude,Longitude,Postal Code
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,M5A
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,M7A
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,M5B
3,Downtown Toronto,St. James Town,43.651494,-79.375418,M5C
4,East Toronto,The Beaches,43.676357,-79.293031,M4E


In [24]:
CLIENT_ID = 'GQLRWSCYD3BBX2KWJEBPYSYZBAGDLTKD5G1A5KBRNUOG4UKU' # your Foursquare ID
CLIENT_SECRET = 'EFE1S3UYDXX2TGFABJSSL1S20MNTLT5QXSUU0N3JO5XTJYKV' # your Foursquare Secret
VERSION = '20200101' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GQLRWSCYD3BBX2KWJEBPYSYZBAGDLTKD5G1A5KBRNUOG4UKU
CLIENT_SECRET:EFE1S3UYDXX2TGFABJSSL1S20MNTLT5QXSUU0N3JO5XTJYKV


use lesson function to help organize Foursquare data

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        LIMIT = 100
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(name, lat, lng, 
                             v['venue']['name'], 
                             v['venue']['location']['lat'], 
                             v['venue']['location']['lng'],  
                             v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                             'Neighborhood Latitude', 
                             'Neighborhood Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

In [26]:
df_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

In [27]:
df_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


shows # of venues returned by Foursquare

In [28]:
df_venues.shape[0]

1614

Tell us the number of different Venue Categories

In [29]:
print ('There are {} different categories'.format(len(df_venues['Venue Category'].unique())))

There are 234 different categories


let's see where all the coffee shops are

In [40]:
df_coffee = df_venues[df_venues['Venue Category'] == 'Coffee Shop']
print(df_coffee.shape)
df_coffee.head()

(144, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
12,"Regent Park, Harbourfront",43.65426,-79.360636,Rooster Coffee,43.651900,-79.365609,Coffee Shop
13,"Regent Park, Harbourfront",43.65426,-79.360636,Starbucks,43.651613,-79.364917,Coffee Shop
15,"Regent Park, Harbourfront",43.65426,-79.360636,Arvo,43.649963,-79.361442,Coffee Shop
19,"Regent Park, Harbourfront",43.65426,-79.360636,Sumach Espresso,43.658135,-79.359515,Coffee Shop


Let's see what the most common coffee shops are

In [51]:
df_coffee.groupby('Venue').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Venue Category
Venue,,,,,,
Aroma Espresso Bar,2,2,2,2,2,2
Arvo,1,1,1,1,1,1
Balzac's Coffee,4,4,4,4,4,4
Balzac’s Coffee Roasters,1,1,1,1,1,1
Bulldog On The Block,4,4,4,4,4,4
Cafe Frappe,1,1,1,1,1,1
Coffee Public,2,2,2,2,2,2
Country Style,1,1,1,1,1,1
Creeds Coffee Bar,1,1,1,1,1,1


We can see there are 42 Starbucks and 22 Tim Hortons (for a total of 64, ignoring the "Tim Hortons / Esso"). Let's map them.

Green dots will be Starbucks, and red dots will be Tim Hortons.

In [61]:
df_starbucks = df_coffee[df_coffee['Venue'] == 'Starbucks']
df_timhortons = df_coffee[df_coffee['Venue'] == 'Tim Hortons']
df_coffee_map = pd.concat([df_starbucks, df_timhortons])
df_coffee_map.reset_index(drop=True, inplace=True)
df_coffee_map.shape

(64, 7)

In [68]:
coffee_map = folium.Map(location=[latitude, longitude])
for line, venue in enumerate(df_coffee_map['Venue']):
    label = '{}, {}'.format(venue, df_coffee_map['Neighborhood'][line])
    label = folium.Popup(label, parse_html=True)
    if venue == 'Starbucks':
        color = 'green'
    else:
        color = 'red'
    folium.CircleMarker([df_coffee_map['Venue Latitude'][line],df_coffee_map['Venue Longitude'][line]], radius=5, popup=label, color=color).add_to(coffee_map)
coffee_map

## Clustering

Let's see where all the coffee is in Toronto, not just Coffee Shops. Anywhere we could expect to find coffee, barring sit-down restaurants

Going through the list of categories, we pull the following: 

'Bagel Shop', 'Bakery', 'Breakfast Spot', 'Café', 'Coffee Shop', 'Creperie', 'Deli / Bodega', 'Dessert Shop', 'Donut Shop', 'Farmers Market', 'Food Court', 'Food & Drink Shop', 'Metro Station', 'Sandwich Place', 'Shopping Mall', 'Train Station'

we make an empty dataframe, df_coffee_all, and populate it with the venue categories above.

In [82]:
categories = ['Bagel Shop', 'Bakery', 'Breakfast Spot', 'Café', 'Coffee Shop', 'Creperie', 'Deli / Bodega', 'Dessert Shop', 'Donut Shop', 'Farmers Market', 'Food Court', 'Food & Drink Shop', 'Metro Station', 'Sandwich Place', 'Shopping Mall', 'Train Station']
df_coffee_all = df_venues[0:2]
df_coffee_all = df_coffee_all[df_coffee_all['Venue'] == 'something_impossible']
for category in categories:
    df_coffee_all = df_coffee_all.append(df_venues[df_venues['Venue Category'] == category])
df_coffee_all.reset_index(drop=True,inplace=True)
df_coffee_all

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,43.644771,-79.373306,St. Urbain Bagel,43.648611,-79.371497,Bagel Shop
1,North Toronto West,43.715383,-79.405678,the bagel house,43.714004,-79.399953,Bagel Shop
2,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,Kiva's,43.687984,-79.394715,Bagel Shop
3,Stn A PO Boxes,43.646435,-79.374846,St. Urbain Bagel,43.648611,-79.371497,Bagel Shop
4,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
...,...,...,...,...,...,...,...
366,"Commerce Court, Victoria Hotel",43.648198,-79.379817,Brookfield Place,43.646791,-79.378769,Shopping Mall
367,Stn A PO Boxes,43.646435,-79.374846,Brookfield Place,43.646791,-79.378769,Shopping Mall
368,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Union Pearson Express,43.644362,-79.383199,Train Station
369,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,Union Pearson Express,43.644362,-79.383199,Train Station


and then we map them:

In [87]:
coffee_map_all = folium.Map(location=[latitude, longitude])
color = 'green'
for line, venue in enumerate(df_coffee_all['Venue']):
    label = '{}, {}, {}'.format(venue, df_coffee_all['Venue Category'][line], df_coffee_all['Neighborhood'][line])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([df_coffee_all['Venue Latitude'][line],df_coffee_all['Venue Longitude'][line]], radius=5, popup=label, color=color).add_to(coffee_map)
coffee_map

Now we cluster them by proximity by using KMeans with lat/lng. We merge this column of data back into the dataframe and remap it. So the difference from the map above is that it's colored by cluster.

In [106]:
from sklearn.cluster import KMeans
kclusters = 10
coffee_clustering = df_coffee_all[['Venue Latitude', 'Venue Longitude']]
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(coffee_clustering)
df_coffee_all['Cluster'] = kmeans.labels_
colors = ['red','blue','gray','darkred','orange','green','darkblue','purple','pink','black']

coffee_map_all = folium.Map(location=[latitude, longitude], zoom_start=12)
for line, venue in enumerate(df_coffee_all['Venue']):
    label = '{}, {}, {}'.format(venue, df_coffee_all['Venue Category'][line], df_coffee_all['Neighborhood'][line])
    label = folium.Popup(label, parse_html=True)
    color = colors[df_coffee_all['Cluster'][line]]
    folium.CircleMarker([df_coffee_all['Venue Latitude'][line],df_coffee_all['Venue Longitude'][line]], radius=5, popup=label, color=color).add_to(coffee_map_all)
coffee_map_all

# mmm Coffee